In [1]:
import pandas as pd
import numpy as np
import json
import ast
from datetime import datetime

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

nos concentramos en el archivo australian_user_reviews.json

In [24]:
rows = []

with open (r'json_originales\australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

reviews = pd.DataFrame(rows)

Este código realiza una operación de desanidación de datos en del DataFrame llamado reviews

In [25]:
# Creamos una nueva lista para almacenar los datos desanidados
data_desanidada = []

# Recorremos cada fila del DataFrame
for index, row in reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    
    # Recorremos cada diccionario en la lista de reviews
    for r in reviews:   
        new_row = {
        'user_id': user_id,
        'user_url': user_url,
        'posted': r.get('posted', ''),
        'item_id': r.get('item_id', ''),
        'recommend': r.get('recommend', ''),
        'review': r.get('review', '')}
        
        data_desanidada.append(new_row)

# Creamos un nuevo DataFrame con los datos desanidados
reviews_desanidado = pd.DataFrame(data_desanidada)

# Ahora df_r_desanidado contiene las columnas 'user_id' y 'user_url' repetidas en los casos necesarios
# junto con los datos desanidados de las reviews

Se cambian las recomendaciones de "True" a 1 y de "False" a 0.

In [26]:
reviews_desanidado.recommend = reviews_desanidado.recommend.astype(int)


Se convierte la columna 'item_id' a tipo de dato entero (int).

In [27]:
reviews_desanidado.item_id = reviews_desanidado.item_id.astype(int)


Se elimina la palabra "Posted" de la columna "posted" para que los valores puedan ser convertidos a formato de fecha. Luego, se convierte la columna "posted" a tipo de dato datetime.

In [28]:
# Convierto la columna 'posted' a tipo cadena
reviews_desanidado['posted'] = reviews_desanidado['posted'].astype(str)
# Realizo el reemplazo
reviews_desanidado['posted'] = reviews_desanidado['posted'].str.replace('Posted ', '')
# Convierto los valores a tipo date
reviews_desanidado['posted'] = pd.to_datetime(reviews_desanidado['posted'], errors='coerce')

Análisis de Sentimientos

In [29]:
reviews_desanidado['review'] = reviews_desanidado['review'].astype(str) # Pasamos a str todos los valores de columna review

Se crea un conjunto de stopwords clásico al que se añaden palabras que se encuentran en igual proporción en recomendaciones positivas y negativas, y que no aportan valor para diferenciar entre categorías.

In [30]:
stopwords = nltk.corpus.stopwords.words('english')
palabras_redundantes = ["game", "play", "make", "good", "hour", "ô", "ñë", "like", "get", "æ", "10",
                    "µ", "one", "would", "time", "really", "å", "fun", "games", "much"]
stopwords.extend(palabras_redundantes)

In [31]:
# Recorremos todos los review y le aplicamos la Normalizacion a cada uno
review_list=[]
for review in reviews_desanidado.review:
    #Sacamos signos de puntuación
    review = re.sub(r'[.,;¡¿?!]', ' ', review)
    #Pasamos a minúsculas
    review = review.lower()
    #Separamos cada palabra de review
    review = word_tokenize(review)
    #Sacamos stopwords
    review = [palabra for palabra in review if not palabra in stopwords]
    # Por ultimo volvemos a unir el review
    review=" ".join(review)

    # Armamos una lista con todos los review
    review_list.append(review)

# Demora 30s

Se crea una columna llamada "review_normalizado" en la que se almacenarán las reseñas después de pasar por el proceso de normalización. La columna "review" será utilizada para categorizar el análisis de sentimiento.

In [32]:
reviews_desanidado['review_normalizado'] = review_list
reviews_desanidado['review'] = 1

Después de analizar cuáles son las palabras utilizadas en recomendaciones negativas y positivas, se crean dos listas y se utilizan para el análisis.

La siguiente lista se puede agrandar para tener un análisis más exhaustivo y personalizado con conocimiento del negocio, incluso incluir distintos idiomas

In [33]:
palabras_positivas = ['π', 'îä', 'great', 'πç', 'best', '3', 'πà', 'amazing', 'πä', 'love', 'ö', 'awesome']
palabras_negativas = ['bad', 'even', 'ç', 'ω', 'buy', 'õ', 'ä', 'ã', 'want', 'people', 'new', '1']

Se le suma 1 punto al review por cada palabra de la lista positiva, se le resta 1 punto por cada palabra de la lista negativa

In [ ]:
for ind, r in enumerate(reviews_desanidado.review_normalizado):
    #Separamos cada palabra de review
    r = word_tokenize(r)
    for nro in range(len(r)):
        if r[nro] in palabras_positivas:
            reviews_desanidado.review.iloc[ind] += 1
        elif r[nro] in palabras_negativas:
            reviews_desanidado.review.iloc[ind] -= 1
    

Se categoriza el sentimiento analizado: negativo = 0, neutral = 1 y positivo = 2

In [35]:
for i in range(len(reviews_desanidado)):
    if reviews_desanidado.loc[i, 'review'] >= 1:
        reviews_desanidado.loc[i, 'review'] = 2
    elif -1 < reviews_desanidado.loc[i, 'review'] < 1:
        reviews_desanidado.loc[i, 'review'] = 1
    else:
        reviews_desanidado.loc[i, 'review'] = 0

# Demora 20s

Nos quedamos únicamente con las columnas que utilizaremos

In [36]:
reviews_desanidado

,user_id,user_url,posted,item_id,recommend,review,review_normalizado
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,1,2,simple yet great replayability opinion `` zomb...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,1,2,'s unique worth playthrough
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,1,2,great atmosphere gunplay bit chunky times end ...
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,1,1,know think see title `` barbie dreamhouse part...
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,1,1,simple ( 's actually simple ) truck driving si...
...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaT,70,1,2,must classic steam definitely worth buying
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaT,362890,1,2,perfect remake original half life personally b...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaT,273110,1,2,plaing collecting resources xd first try kille...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaT,730,1,2,:


In [37]:
reviews_desanidado = reviews_desanidado[['user_id','user_url','posted','item_id', 'recommend', 'review']]


La exportación como archivo CSV del DataFrame 'reviews_desanidado' se realiza de la siguiente manera.

In [41]:
reviews_desanidado.to_csv(r'csv_ETL\reviews_desanidado.csv', index=False)

In [39]:
reviews_desanidado.shape

(59305, 6)

In [40]:
reviews_desanidado

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,1,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,1,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,1,2
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,1,1
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,1,1
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaT,70,1,2
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaT,362890,1,2
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaT,273110,1,2
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaT,730,1,2
